<a href="https://colab.research.google.com/github/svhenrique/house-prices-predictor/blob/main/HousePricesPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando Bibliotécas necessárias

In [66]:
# montando drive (para o colab)
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [67]:
# importando bibliotecas necessárias
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## Lendo dados

In [68]:
# dados em pasta 
pasta = '/content/drive/My Drive/datasets/house-prices/'

In [69]:
# lendo dados de treino e teste
train_data = pd.read_csv(pasta + 'train.csv')
test_data = pd.read_csv(pasta + 'test.csv')

## Limpeza e organização dos dados

In [70]:
# mostrando shape resultante
print('Train data shape: ', train_data.shape)
print('Test data shape: ', test_data.shape)

Train data shape:  (1460, 81)
Test data shape:  (1459, 80)


### Limpando dados

In [71]:
# removendo dados nan de train_data
train_data.dropna(how='any',axis=0)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [72]:
# removendo dados nan de test_data
test_data.dropna(how='any',axis=0)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


É possível observar que todas as linhas foram apagadas, isso indica que existem valores "nan" em todas as instâncias. Entretanto, isso é uma confusão que o pandas fez ao ler o atributo "NA" de "Alley", que significa "No alley access".

In [73]:
# observando as features com maiores ocorrências de nan 
train_data.isnull().sum()[train_data.isnull().sum() > len(train_data)/2]

Alley          1369
PoolQC         1453
Fence          1179
MiscFeature    1406
dtype: int64

Logo, essas 4 features podem ter valores normais que são lidos como nan.

Eu irei seguir o glossário de valores para identificar atributos como "NA" que realmente significam algo, evitando perca de dados.

In [74]:
# Existem valores NA, que são confundidos com nans, sobre nas colunas a seguir
print(train_data['Alley'].unique())
print(train_data['PoolQC'].unique())
print(train_data['Fence'].unique())
print(train_data['MiscFeature'].unique())
print(train_data['BsmtQual'].unique())
print(train_data['BsmtCond'].unique())
print(train_data['BsmtExposure'].unique())
print(train_data['BsmtFinType1'].unique())
print(train_data['BsmtFinType2'].unique())
print(train_data['FireplaceQu'].unique())
print(train_data['GarageType'].unique())
print(train_data['GarageFinish'].unique())
print(train_data['GarageQual'].unique())
print(train_data['GarageCond'].unique())

[nan 'Grvl' 'Pave']
[nan 'Ex' 'Fa' 'Gd']
[nan 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']
[nan 'Shed' 'Gar2' 'Othr' 'TenC']
['Gd' 'TA' 'Ex' nan 'Fa']
['TA' 'Gd' nan 'Fa' 'Po']
['No' 'Gd' 'Mn' 'Av' nan]
['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
[nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']
['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
['RFn' 'Unf' 'Fin' nan]
['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
['TA' 'Fa' nan 'Gd' 'Po' 'Ex']


In [75]:
# inserindo o valor "NA" (No alley access) para os nan em Alley de treino e teste
train_data['Alley'] = train_data['Alley'].fillna('NA')
test_data['Alley'] = test_data['Alley'].fillna('NA')

In [76]:
# inserindo o valor "NA" (No Pool) para os nan em Alley de treino e teste
train_data['PoolQC'] = train_data['PoolQC'].fillna('NA')
test_data['PoolQC'] = test_data['PoolQC'].fillna('NA')

In [77]:
# inserindo o valor "NA" (No Fencel) para os nan em PoolQC de treino e teste
train_data['Fence'] = train_data['Fence'].fillna('NA')
test_data['Fence'] = test_data['Fence'].fillna('NA')

In [78]:
# inserindo o valor "NA" (None) para os nan em PoolQC de treino e teste
train_data['MiscFeature'] = train_data['MiscFeature'].fillna('NA')
test_data['MiscFeature'] = test_data['MiscFeature'].fillna('NA')

In [79]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtQual de treino e teste
train_data['BsmtQual'] = train_data['BsmtQual'].fillna('NA')
test_data['BsmtQual'] = test_data['BsmtQual'].fillna('NA')

In [80]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtCond de treino e teste
train_data['BsmtCond'] = train_data['BsmtCond'].fillna('NA')
test_data['BsmtCond'] = test_data['BsmtCond'].fillna('NA')

In [81]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtExposure de treino e teste
train_data['BsmtExposure'] = train_data['BsmtExposure'].fillna('NA')
test_data['BsmtExposure'] = test_data['BsmtExposure'].fillna('NA')

In [82]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtFinType1 de treino e teste
train_data['BsmtFinType1'] = train_data['BsmtFinType1'].fillna('NA')
test_data['BsmtFinType1'] = test_data['BsmtFinType1'].fillna('NA')

In [83]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtFinType2 de treino e teste
train_data['BsmtFinType2'] = train_data['BsmtFinType2'].fillna('NA')
test_data['BsmtFinType2'] = test_data['BsmtFinType2'].fillna('NA')

In [84]:
# inserindo o valor "NA" (No Fireplace) para os nan em FireplaceQu de treino e teste
train_data['FireplaceQu'] = train_data['FireplaceQu'].fillna('NA')
test_data['FireplaceQu'] = test_data['FireplaceQu'].fillna('NA')

In [85]:
# inserindo o valor "NA" (No Garage) para os nan em GarageType de treino e teste
train_data['GarageType'] = train_data['GarageType'].fillna('NA')
test_data['GarageType'] = test_data['GarageType'].fillna('NA')

In [86]:
# inserindo o valor "NA" (No Garage) para os nan em GarageFinish de treino e teste
train_data['GarageFinish'] = train_data['GarageFinish'].fillna('NA')
test_data['GarageFinish'] = test_data['GarageFinish'].fillna('NA')

In [87]:
# inserindo o valor "NA" (No Garage) para os nan em GarageQual de treino e teste
train_data['GarageQual'] = train_data['GarageQual'].fillna('NA')
test_data['GarageQual'] = test_data['GarageQual'].fillna('NA')

In [88]:
# inserindo o valor "NA" (No Garage) para os nan em GarageCond de treino e teste
train_data['GarageCond'] = train_data['GarageCond'].fillna('NA')
test_data['GarageCond'] = test_data['GarageCond'].fillna('NA')

In [89]:
# removendo os dados nan verdadeiros dos datasets
train_data = train_data.dropna(how='any',axis=0)
test_data = test_data.dropna(how='any',axis=0)

In [90]:
# mostrando shape resultante
print('Train data shape: ', train_data.shape)
print('Test data shape: ', test_data.shape)

Train data shape:  (1120, 81)
Test data shape:  (1139, 80)


In [91]:
# removendo coluna id (pois ela é inútil para a criação do modelo) 
train_data.drop('Id', inplace=True, axis=1)
test_data.drop('Id', inplace=True, axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Organizando dados

In [92]:
train_data.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [93]:
column = 'MSZoning'
print(list(train_data[column].unique()))
print(list(test_data[column].unique()))

['RL', 'RM', 'C (all)', 'FV', 'RH']
['RH', 'RL', 'RM', 'FV', 'C (all)']


In [94]:
# transformando dados categóricos em dados numéricos

columns = list(train_data.columns)
columns.remove('SalePrice')

for column in columns:

  
  data1 = train_data[column].unique()
  data2 = list(test_data[column].unique())

  # pegando valores diferentes dos dois datasets e juntando
  for data in data2:
    if data not in data1:
      data1 = np.append(data1, data)

  if data1.dtype not in [np.int64, np.float64]:
    simbols = list(range(data1.shape[0]))
    print()
    print(data1)
    print(simbols)
    print()

    train_data.replace(data1, simbols, inplace=True)
    test_data.replace(data1, simbols, inplace=True)


['RL' 'RM' 'C (all)' 'FV' 'RH']
[0, 1, 2, 3, 4]


['Pave' 'Grvl']
[0, 1]


['NA' 1 0]
[0, 1, 2]


['Reg' 'IR1' 'IR2' 'IR3']
[0, 1, 2, 3]


['Lvl' 'Bnk' 'Low' 'HLS']
[0, 1, 2, 3]


['AllPub']
[0]


['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
[0, 1, 2, 3, 4]


['Gtl' 'Mod' 'Sev']
[0, 1, 2]


['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'OldTown'
 'BrkSide' 'Sawyer' 'NridgHt' 'SawyerW' 'NAmes' 'IDOTRR' 'MeadowV'
 'Timber' 'StoneBr' 'ClearCr' 'Gilbert' 'Edwards' 'NWAmes' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


['Norm' 'Feedr' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosN' 'PosA' 'RRNe']
[0, 1, 2, 3, 4, 5, 6, 7, 8]


['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
[0, 1, 2, 3, 4]


['2Story' '1Story' '1.5Fin' '1.5Unf' 'SLvl' '2.5Unf' '2.5Fin' 'SFoyer']
[0, 1, 2, 3, 4, 5, 6, 7]


['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
[0, 1, 2, 3, 4, 5]


['CompShg' 'WdShngl' 'Membran' 'WdShake' 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':



['Typ' 'Min1' 'Maj1' 'Min2' 4 'Maj2']
[0, 1, 2, 3, 4, 5]


['Attchd' 'Detchd' 'BuiltIn' 'CarPort' 'Basment' '2Types']
[0, 1, 2, 3, 4, 5]


['RFn' 2 'Fin']
[0, 1, 2]


[0 1 'P']
[0, 1, 2]


[4 'MnPrv' 'GdPrv' 'GdWo' 'MnWw']
[0, 1, 2, 3, 4]


[0 5 'Othr' 'TenC' 'Gar2']
[0, 1, 2, 3, 4]


['WD' 'New' 'COD' 'ConLI' 'CWD' 'ConLw' 'Con' 'ConLD' 'Oth']
[0, 1, 2, 3, 4, 5, 6, 7, 8]


['Normal' 'Abnorml' 'Partial' 'Alloca' 'Family' 'AdjLand']
[0, 1, 2, 3, 4, 5]

